# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [1]:
import getpass
myname = getpass.getuser().split('-')[1]
myname
MYTABLE=myname + '_data'



In [2]:
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [3]:
host = 'slucor2022-instance-1.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2022'
password = 'SLUcor2022'
database = 'hds5210'
conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)

In [4]:
assert(MYTABLE != 'pboal')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [5]:
data="https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326"
ehr=pd.read_csv(data)
ehr

,X,Y,OBJECTID,Provider_Name,NPI,CCN,Business_Street_Address,Business_City,Business_County,Business_ZIP_Code,...,Payment_Year_Number,Program_Type,Medicaid_EP_Hospital_Type,total_payments,Last_Payment_Criteria,Recent_Disbursement_Amount,Latitude,Longitude,Last_Program_Year,Last_Payment_Year
0,-124.142009,40.783559,1,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1609858950,50006,2700 Dolbeer St,Eureka,Humboldt,95501,...,4,Medicare/Medicaid,Acute Care Hospitals,1530950.70,MU,153095.07,40.783559,-124.142009,2015,2016
1,-122.086674,37.632915,2,HAYWARD SISTERS HOSPITAL,1942298153,50002,27200 Calaroga Ave,Hayward,Alameda,94545,...,4,Medicare/Medicaid,Acute Care Hospitals,3245920.28,MU,324592.03,37.632915,-122.086674,2015,2016
2,-122.295861,38.325402,3,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1235218785,50009,1000 Trancas St,Napa,Napa,94558,...,4,Medicare/Medicaid,Acute Care Hospitals,1262015.89,MU,126201.59,38.325402,-122.295861,2015,2016
3,-120.766467,38.350920,4,SUTTER VALLEY HOSPITALS,1447494323,50014,200 Mission Blvd,Jackson,Amador,95642,...,4,Medicare/Medicaid,Acute Care Hospitals,865130.13,MU,86513.01,38.350920,-120.766467,2017,2019
4,-122.435820,37.769049,5,Sutter Bay Hospitals,1659439834,50008,CASTRO AND DUBOCE,San Francisco,San Francisco,94114,...,4,Medicare/Medicaid,Acute Care Hospitals,984465.12,MU,98446.51,37.769049,-122.435820,2016,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,-122.173214,37.436328,319,LUCILE SALTER PACKARD CHILDREN'S HOSPITAL AT S...,1467442749,53305,"725 Welch Rd, mail code 5500",Palo Alto,Santa Clara,94304,...,4,Medicaid,Children's Hospitals,5630849.63,MU,563084.96,37.436328,-122.173214,2017,2019
319,-118.186632,33.808293,320,LONG BEACH MEMORIAL MEDICAL,1477596583,53309,2801 Atlantic Ave,Long Beach,Los Angeles,90806,...,4,Medicaid,Children's Hospitals,7951873.30,MU,795187.33,33.808293,-118.186632,2015,2016
320,-121.455450,38.553012,321,SHRINERS HOSPITALS FOR CHILDREN,1962530451,53311,2425 Stockton Blvd,Sacramento,Sacramento,95817,...,4,Medicaid,Children's Hospitals,2184269.03,MU,218426.90,38.553012,-121.455450,2018,2020
321,-118.286401,34.067178,322,SHRINERS HOSPITALS FOR CHILDREN,1508908591,53310,3160 Geneva Street,Los Angeles,Los Angeles,90020,...,1,Medicaid,Children's Hospitals,427640.22,AIU,427640.22,34.067178,-118.286401,2016,2017


In [6]:
ehr.to_sql(myname + '_data', conn, index=True,if_exists='replace')

In [7]:
ehr.shape

(323, 21)

In [8]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
dxyz.shape

(323, 22)

In [9]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (323,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'])

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Business_ZIP_Code` field and `population`'s `Zip` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [10]:
df = pd.read_sql_query("SHOW TABLES", conn)

In [11]:
df

,Tables_in_hds5210
0,asahu123_data
1,asahu123_ipv
2,data
3,ecdmdphd_data
4,ed_visits
5,havenaspen_data
6,havenaspen_ipv
7,hospital_services
8,hospitals
9,lillyeversman_data


In [12]:
tables = {}

for n,t in pd.read_sql_query("SHOW TABLES", conn).iterrows():
    name = str(t['Tables_in_hds5210'])
    print(name)
    tables[name] = pd.read_sql("SELECT * FROM "+name,conn)

asahu123_data
asahu123_ipv
data
ecdmdphd_data
ed_visits
havenaspen_data
havenaspen_ipv
hospital_services
hospitals
lillyeversman_data
paulboal_data
payment_data
pbstudent_data
population
rasalt_data
rasalt_data_tmp
rasalt_data_tmp1
rasalt_data_tmp2
sravani9544_data
svelicheti_data
system_affiliations
total_per_population


In [13]:
tables['havenaspen_data'].shape

(323, 22)

In [14]:
tables['population']

,index,Zip,Population
0,0,90011,110750
1,1,90650,104524
2,2,91331,103683
3,3,90201,100512
4,4,90044,99980
...,...,...,...
1645,1645,95699,13
1646,1646,96119,11
1647,1647,92304,7
1648,1648,93530,6


In [15]:
dhcs=pd.merge(tables['population'],tables['havenaspen_data'],how='right',left_on='Zip', right_on='Business_ZIP_Code')
dhcs


,index_x,Zip,Population,index_y,X,Y,OBJECTID,Provider_Name,NPI,CCN,...,Payment_Year_Number,Program_Type,Medicaid_EP_Hospital_Type,total_payments,Last_Payment_Criteria,Recent_Disbursement_Amount,Latitude,Longitude,Last_Program_Year,Last_Payment_Year
0,736.0,95501.0,23948.0,0,-124.142009,40.783559,1,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1609858950,50006,...,4,Medicare/Medicaid,Acute Care Hospitals,1530950.70,MU,153095.07,40.783559,-124.142009,2015,2016
1,543.0,94545.0,32552.0,1,-122.086674,37.632915,2,HAYWARD SISTERS HOSPITAL,1942298153,50002,...,4,Medicare/Medicaid,Acute Care Hospitals,3245920.28,MU,324592.03,37.632915,-122.086674,2015,2016
2,543.0,94545.0,32552.0,194,-122.096165,37.630272,195,KAISER FOUNDATION HOSPITALS,1801960513,50512,...,4,Medicare/Medicaid,Acute Care Hospitals,415091.50,MU,41509.15,37.630272,-122.096165,2018,2020
3,77.0,94558.0,66854.0,2,-122.295861,38.325402,3,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1235218785,50009,...,4,Medicare/Medicaid,Acute Care Hospitals,1262015.89,MU,126201.59,38.325402,-122.295861,2015,2016
4,1067.0,95642.0,6547.0,3,-120.766467,38.350920,4,SUTTER VALLEY HOSPITALS,1447494323,50014,...,4,Medicare/Medicaid,Acute Care Hospitals,865130.13,MU,86513.01,38.350920,-120.766467,2017,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,972.0,95453.0,11209.0,312,-122.907916,39.105474,313,Sutter Bay Hospitals,1952634008,51329,...,4,Medicare/Medicaid,Acute Care Hospitals,1142752.33,MU,114275.23,39.105474,-122.907916,2016,2019
319,741.0,94609.0,23729.0,313,-122.267252,37.837338,314,Children's Hospital & Research Center at Oakland,1003961251,53301,...,4,Medicaid,Children's Hospitals,6760775.32,MU,676077.53,37.837338,-122.267252,2016,2019
320,888.0,92866.0,15428.0,316,-117.833859,33.782715,317,HEALTHBRIDGE CHILDREN'S HOSPITAL-ORANGE CA LP,1225015985,53308,...,2,Medicaid,Children's Hospitals,1543558.29,MU,578834.36,33.782715,-117.833859,2015,2017
321,1133.0,94304.0,4391.0,318,-122.173214,37.436328,319,LUCILE SALTER PACKARD CHILDREN'S HOSPITAL AT S...,1467442749,53305,...,4,Medicaid,Children's Hospitals,5630849.63,MU,563084.96,37.436328,-122.173214,2017,2019


In [16]:
df=dhcs.loc[:, ['index_x','NPI','Zip','Population']]
df

,index_x,NPI,Zip,Population
0,736.0,1609858950,95501.0,23948.0
1,543.0,1942298153,94545.0,32552.0
2,543.0,1801960513,94545.0,32552.0
3,77.0,1235218785,94558.0,66854.0
4,1067.0,1447494323,95642.0,6547.0
...,...,...,...,...
318,972.0,1952634008,95453.0,11209.0
319,741.0,1003961251,94609.0,23729.0
320,888.0,1225015985,92866.0,15428.0
321,1133.0,1467442749,94304.0,4391.0


In [17]:
df['Population']==df['Population'].max()
df

,index_x,NPI,Zip,Population
0,736.0,1609858950,95501.0,23948.0
1,543.0,1942298153,94545.0,32552.0
2,543.0,1801960513,94545.0,32552.0
3,77.0,1235218785,94558.0,66854.0
4,1067.0,1447494323,95642.0,6547.0
...,...,...,...,...
318,972.0,1952634008,95453.0,11209.0
319,741.0,1003961251,94609.0,23729.0
320,888.0,1225015985,92866.0,15428.0
321,1133.0,1467442749,94304.0,4391.0


In [18]:
max_provider_npi=df[df['Population']==df['Population'].max()]['NPI']
max_provider_npi

286    1194016923
Name: NPI, dtype: int64

In [19]:
answer = max_provider_npi.tolist()
answer[0] = str(answer[0])

In [20]:
assert(type(answer) == list)
assert(answer == ['1194016923'])


## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

## First using SQL

In [21]:
total_payment=pd.read_sql_query("""
  SELECT m.total_payments, m.Business_ZIP_code, m.Medicaid_EP_Hospital_Type, p.Population, p.Zip
FROM 
    havenaspen_data m INNER JOIN
    population p ON m.business_zip_code = p.zip
GROUP BY Zip 
""", conn)
total_payment

,total_payments,Business_ZIP_code,Medicaid_EP_Hospital_Type,Population,Zip
0,1093021.73,90004,Acute Care Hospitals,61105,90004
1,3614739.93,90012,Acute Care Hospitals,37268,90012
2,10045484.04,90015,Acute Care Hospitals,23900,90015
3,3899845.21,90017,Acute Care Hospitals,27832,90017
4,427640.22,90020,Children's Hospitals,39366,90020
...,...,...,...,...,...
251,184615.39,96104,Acute Care Hospitals,920,96104
252,531907.08,96122,Acute Care Hospitals,3594,96122
253,1371962.88,96130,Acute Care Hospitals,19859,96130
254,786769.13,96150,Acute Care Hospitals,30369,96150


In [22]:
total_payment['personal_payment']=total_payment['total_payments']/total_payment['Population']
total_payment

,total_payments,Business_ZIP_code,Medicaid_EP_Hospital_Type,Population,Zip,personal_payment
0,1093021.73,90004,Acute Care Hospitals,61105,90004,17.887599
1,3614739.93,90012,Acute Care Hospitals,37268,90012,96.993129
2,10045484.04,90015,Acute Care Hospitals,23900,90015,420.313140
3,3899845.21,90017,Acute Care Hospitals,27832,90017,140.120912
4,427640.22,90020,Children's Hospitals,39366,90020,10.863187
...,...,...,...,...,...,...
251,184615.39,96104,Acute Care Hospitals,920,96104,200.668902
252,531907.08,96122,Acute Care Hospitals,3594,96122,147.998631
253,1371962.88,96130,Acute Care Hospitals,19859,96130,69.085195
254,786769.13,96150,Acute Care Hospitals,30369,96150,25.906982


In [23]:
answer=total_payment.loc[:, ['Medicaid_EP_Hospital_Type','personal_payment']]
answer

,Medicaid_EP_Hospital_Type,personal_payment
0,Acute Care Hospitals,17.887599
1,Acute Care Hospitals,96.993129
2,Acute Care Hospitals,420.313140
3,Acute Care Hospitals,140.120912
4,Children's Hospitals,10.863187
...,...,...
251,Acute Care Hospitals,200.668902
252,Acute Care Hospitals,147.998631
253,Acute Care Hospitals,69.085195
254,Acute Care Hospitals,25.906982


In [24]:
ach=answer.loc[answer['Medicaid_EP_Hospital_Type'] == "Acute Care Hospitals"]["personal_payment"].sum() / len(answer.loc[answer['Medicaid_EP_Hospital_Type'] == "Acute Care Hospitals"])
ch=answer.loc[answer['Medicaid_EP_Hospital_Type'] == "Children's Hospitals"]["personal_payment"].sum() / len(answer.loc[answer['Medicaid_EP_Hospital_Type'] == "Children's Hospitals"])
answer.loc[answer['Medicaid_EP_Hospital_Type'] == "Acute Care Hospitals", ["Avg_Pay_per_Capita"]] = ach
answer.loc[answer['Medicaid_EP_Hospital_Type'] == "Children's Hospitals", ["Avg_Pay_per_Capita"]] = ch
answer=answer.loc[:, ['Medicaid_EP_Hospital_Type','Avg_Pay_per_Capita']]

answer

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,87.035200
1,Acute Care Hospitals,87.035200
2,Acute Care Hospitals,87.035200
3,Acute Care Hospitals,87.035200
4,Children's Hospitals,468.554016
...,...,...
251,Acute Care Hospitals,87.035200
252,Acute Care Hospitals,87.035200
253,Acute Care Hospitals,87.035200
254,Acute Care Hospitals,87.035200


In [25]:
answer.drop_duplicates()

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,87.035200
4,Children's Hospitals,468.554016


In [26]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 106.583)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 329.551)


AssertionError: 

In [ ]:
answer

## Now using Pandas instead of SQL

In [ ]:
dhcs=pd.merge(tables['population'],tables['havenaspen_data'],how='right',left_on='Zip', right_on='Business_ZIP_Code')
dhcs


In [ ]:
df=dhcs.loc[:, ['total_payments','Medicaid_EP_Hospital_Type','Population','Zip']]
df

In [ ]:
df['personal_payment']=df['total_payments']/df['Population']
answer=df
answer

In [ ]:
ach=answer.loc[answer['Medicaid_EP_Hospital_Type'] == "Acute Care Hospitals"]["personal_payment"].sum() / len(answer.loc[answer['Medicaid_EP_Hospital_Type'] == "Acute Care Hospitals"])
ch=answer.loc[answer['Medicaid_EP_Hospital_Type'] == "Children's Hospitals"]["personal_payment"].sum() / len(answer.loc[answer['Medicaid_EP_Hospital_Type'] == "Children's Hospitals"])
answer.loc[answer['Medicaid_EP_Hospital_Type'] == "Acute Care Hospitals", ["Avg_Pay_per_Capita"]] = ach
answer.loc[answer['Medicaid_EP_Hospital_Type'] == "Children's Hospitals", ["Avg_Pay_per_Capita"]] = ch
answer=answer.loc[:, ['Medicaid_EP_Hospital_Type','Avg_Pay_per_Capita']]

answer

In [ ]:
answer.drop_duplicates()



---



## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [27]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git pull
    !git add week15_assignment.ipynb
    !git commit -a -m "Submitting the week 15 programming assignment"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


Already up to date.
[main 61a3de1] Submitting the week 15 programming assignment
 1 file changed, 2645 insertions(+)
 create mode 100644 week15/week15_assignment.ipynb
Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 11.16 KiB | 5.58 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
remote: This repository moved. Please use the new location:
remote:   git@github.com:HavenAspen/hds5210-2022.git
To github.com:havenaspen/hds5210-2022.git
   b77e188..61a3de1  main -> main
